In [ ]:
import tkinter as tk
from tkinter import messagebox
from tkinter import ttk


# Backend Classes
class Ticket:
    def __init__(self, ticket_type, price, validity, discount=0):
        self.ticket_type = ticket_type
        self.price = price
        self.validity = validity
        self.discount = discount

    def calculate_price(self, quantity):
        total = self.price * quantity
        if self.discount > 0:
            total -= total * (self.discount / 100)
        return total


class User:
    def __init__(self, username, email):
        self.username = username
        self.email = email
        self.purchase_history = []

    def add_purchase(self, ticket, quantity, total_price):
        purchase = {
            "ticket_type": ticket.ticket_type,
            "quantity": quantity,
            "total_price": total_price
        }
        self.purchase_history.append(purchase)

    def view_purchase_history(self):
        return self.purchase_history


class Admin:
    def __init__(self):
        self.ticket_sales = {}

    def record_sale(self, ticket_type, quantity):
        if ticket_type in self.ticket_sales:
            self.ticket_sales[ticket_type] += quantity
        else:
            self.ticket_sales[ticket_type] = quantity

    def view_sales(self):
        return self.ticket_sales


class TicketBookingSystem:
    def __init__(self):
        self.tickets = []
        self.users = {}
        self.admin = Admin()

    def add_ticket(self, ticket):
        self.tickets.append(ticket)

    def register_user(self, username, email):
        if username not in self.users:
            self.users[username] = User(username, email)
        else:
            print("User already exists!")

    def purchase_ticket(self, username, ticket_type, quantity):
        user = self.users.get(username)
        if user:
            for ticket in self.tickets:
                if ticket.ticket_type == ticket_type:
                    total_price = ticket.calculate_price(quantity)
                    user.add_purchase(ticket, quantity, total_price)
                    self.admin.record_sale(ticket_type, quantity)
                    print(f"Purchase successful! Total price: {total_price} DHS")
                    return
            print("Ticket type not found!")
        else:
            print("User not registered!")


# GUI Application
class TicketBookingApp:
    def __init__(self, root, system):
        self.root = root
        self.system = system
        self.root.title("Adventure Land Ticket Booking")
        self.root.geometry("600x500")

        self.notebook = ttk.Notebook(self.root)
        self.notebook.pack(expand=True, fill="both")

        self.create_user_tab()
        self.create_ticket_tab()
        self.create_admin_tab()

    def create_user_tab(self):
        user_tab = ttk.Frame(self.notebook)
        self.notebook.add(user_tab, text="User Section")

        # User Registration
        tk.Label(user_tab, text="Register User").grid(row=0, column=0, pady=10)
        tk.Label(user_tab, text="Username:").grid(row=1, column=0)
        tk.Label(user_tab, text="Email:").grid(row=2, column=0)

        self.username_entry = tk.Entry(user_tab)
        self.username_entry.grid(row=1, column=1)
        self.email_entry = tk.Entry(user_tab)
        self.email_entry.grid(row=2, column=1)

        tk.Button(user_tab, text="Register", command=self.register_user).grid(row=3, column=1, pady=10)

        # Ticket Purchase
        tk.Label(user_tab, text="Purchase Ticket").grid(row=4, column=0, pady=10)
        tk.Label(user_tab, text="Username:").grid(row=5, column=0)
        tk.Label(user_tab, text="Ticket Type:").grid(row=6, column=0)
        tk.Label(user_tab, text="Quantity:").grid(row=7, column=0)

        self.purchase_username_entry = tk.Entry(user_tab)
        self.purchase_username_entry.grid(row=5, column=1)
        self.ticket_type_combobox = ttk.Combobox(user_tab, values=[t.ticket_type for t in self.system.tickets])
        self.ticket_type_combobox.grid(row=6, column=1)
        self.quantity_entry = tk.Entry(user_tab)
        self.quantity_entry.grid(row=7, column=1)

        tk.Button(user_tab, text="Purchase", command=self.purchase_ticket).grid(row=8, column=1, pady=10)

    def create_ticket_tab(self):
        ticket_tab = ttk.Frame(self.notebook)
        self.notebook.add(ticket_tab, text="Tickets & History")

        # View Purchase History
        tk.Label(ticket_tab, text="View Purchase History").grid(row=0, column=0, pady=10)
        tk.Label(ticket_tab, text="Username:").grid(row=1, column=0)

        self.history_username_entry = tk.Entry(ticket_tab)
        self.history_username_entry.grid(row=1, column=1)

        tk.Button(ticket_tab, text="View", command=self.view_purchase_history).grid(row=2, column=1, pady=10)

        self.history_display = tk.Text(ticket_tab, height=15, width=50)
        self.history_display.grid(row=3, column=0, columnspan=2, pady=10)

    def create_admin_tab(self):
        admin_tab = ttk.Frame(self.notebook)
        self.notebook.add(admin_tab, text="Admin Dashboard")

        tk.Label(admin_tab, text="Admin Dashboard").grid(row=0, column=0, pady=10)
        tk.Button(admin_tab, text="View Sales", command=self.view_sales).grid(row=1, column=0, pady=10)

        self.sales_display = tk.Text(admin_tab, height=15, width=50)
        self.sales_display.grid(row=2, column=0, pady=10)

    def register_user(self):
        username = self.username_entry.get()
        email = self.email_entry.get()
        if username and email:
            self.system.register_user(username, email)
            messagebox.showinfo("Success", f"User '{username}' registered successfully!")
        else:
            messagebox.showerror("Error", "Username and email are required!")

    def purchase_ticket(self):
        username = self.purchase_username_entry.get()
        ticket_type = self.ticket_type_combobox.get()
        quantity = self.quantity_entry.get()
        if username and ticket_type and quantity.isdigit():
            quantity = int(quantity)
            self.system.purchase_ticket(username, ticket_type, quantity)
            messagebox.showinfo("Success", f"Ticket(s) purchased successfully!")
        else:
            messagebox.showerror("Error", "All fields are required and quantity must be a number!")

    def view_purchase_history(self):
        username = self.history_username_entry.get()
        if username in self.system.users:
            user = self.system.users[username]
            history = user.view_purchase_history()
            self.history_display.delete(1.0, tk.END)
            for record in history:
                self.history_display.insert(tk.END, f"Ticket: {record['ticket_type']}, Quantity: {record['quantity']}, Total: {record['total_price']} DHS\n")
        else:
            messagebox.showerror("Error", "User not found!")

    def view_sales(self):
        sales = self.system.admin.view_sales()
        self.sales_display.delete(1.0, tk.END)
        for ticket_type, quantity in sales.items():
            self.sales_display.insert(tk.END, f"Ticket: {ticket_type}, Sold: {quantity}\n")


# Initialize the backend system
system = TicketBookingSystem()
system.add_ticket(Ticket("Single-Day Pass", 275, "1 Day"))
system.add_ticket(Ticket("Two-Day Pass", 480, "2 Days", 10))
system.add_ticket(Ticket("Annual Membership", 1840, "1 Year", 15))
system.add_ticket(Ticket("Group Ticket", 220, "1 Day", 20))
system.add_ticket(Ticket("Child Ticket", 185, "1 Day"))

# Run the GUI
root = tk.Tk()
app = TicketBookingApp(root, system)
root.mainloop()